In [54]:
%load_ext autoreload
%autoreload 2
import data_loader
from feature_extraction import extract_features, data_in_tw
import matplotlib.pyplot as plt

#from datetime import datetime, timedelta
import pandas as pd
import pytz
import pickle

from sklearn.svm import SVC

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample

from sklearn.metrics import mean_squared_error
import numpy as np




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# Size of time window in unit of second.
TIME_WINDOW_SEC = 20
features = []
labels = []


# Data Loading

In [56]:
dl = data_loader.DataLoader()

# Load Groudtruth
df_labels = dl.load_groundtruth()


# Load low frequency data
df_location = dl.load_low_freq_data('LOCATION')
df_activity = dl.load_low_freq_data('ACTIVITY_TYPE')



Loading /Users/zxxia/Research/bharath-data/lowfreqdata/10_null_LOCATION_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...
Loading /Users/zxxia/Research/bharath-data/lowfreqdata/11_null_ACTIVITY_TYPE_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...


In [57]:
visited = set()
_, type_dsid_dict, beacon_id_set = dl.load_datasources()

for index, row in df_labels.iterrows():
    d = row['Start Timestamp'].date()
    if d not in visited:
        # load high frequency data
        df_battery = dl.load_high_freq_data('BATTERY', d)
        # df_compass = dl.load_high_freq_data('COMPASS', d)
        df_ambient_light = dl.load_high_freq_data('AMBIENT_LIGHT', d)
        df_car_beacon = dl.load_high_freq_data('BEACON_CAR', d)
        df_home_beacon = dl.load_high_freq_data('BEACON_Living Room', d)
        df_office_beacon = None
        df_work1_beacon = dl.load_high_freq_data('BEACON_Laptop', d)
        df_work2_beacon = None # dl.load_high_freq_data('BEACON_Laptop', d)
        visited.add(d)

    start = row['Start Timestamp']
    end = row['End Timestamp']

    if start >= end:
        continue

    # Obtain data needed for one label(one event)
    streams = dict()
    streams['Battery'] = data_in_tw(df_battery, start, end)
    streams['Compass'] = data_in_tw(df_compass, start, end)
    streams['Activity'] = data_in_tw(df_activity, start, end)
    streams['Location'] = data_in_tw(df_location, start, end)
    streams['Ambient Light'] = data_in_tw(df_ambient_light, start, end)
    streams['Car Beacon'] = data_in_tw(df_beacon, start, end)
    streams['Home Beacon'] = data_in_tw(df_beacon, start, end)
    streams['Work1 Beacon'] = data_in_tw(df_beacon, start, end)
    streams['Work2 Beacon'] = data_in_tw(df_beacon, start, end)
    streams['Office Beacon'] = data_in_tw(df_beacon, start, end)

    # Segment data streams into time windows, extract features and append features
    tmp = extract_features(streams, start, end, TIME_WINDOW_SEC)
    features = features + tmp
    
    # Extract labels and append labels
    labels = labels + [row['Labels']] * len(tmp)

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012000_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012001_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012002_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012003_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012004_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012005_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012006_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012007_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012008_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012009_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012010_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012011_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012012_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012200_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012201_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012202_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012203_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012204_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012205_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012206_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012207_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012208_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012209_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012210_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012211_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012212_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012305_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012306_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012307_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012308_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012309_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012310_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012311_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012312_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012323_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012300_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012301_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012302_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw22

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012509_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012510_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012511_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012512_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012513_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012514_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012515_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012516_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012517_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012518_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012519_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw15/2018012501_15.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw15/2018012509_15.csv.gz...
Loading /Users/zxxia/

In [61]:
print(len(features))
print(len(labels))
display(features)


22714
22714


[{'Battery Charging': nan,
  'Car Beacon': 0,
  'IN_VEHICLE': 0,
  'Indoor Beacon': 0,
  'Max GPS Accuracy': nan,
  'Max GPS Speed': nan,
  'Max Light Intensity': nan,
  'ON_BICYCLE': 0,
  'ON_FOOT': 0,
  'RUNNING': 0,
  'STILL': 1,
  'TILTING': 0,
  'Time of day': 0,
  'UNKNOWN': 0,
  'WALKING': 0},
 {'Battery Charging': nan,
  'Car Beacon': 0,
  'IN_VEHICLE': 0,
  'Indoor Beacon': 0,
  'Max GPS Accuracy': nan,
  'Max GPS Speed': nan,
  'Max Light Intensity': nan,
  'ON_BICYCLE': 0,
  'ON_FOOT': 0,
  'RUNNING': 0,
  'STILL': 1,
  'TILTING': 0,
  'Time of day': 0,
  'UNKNOWN': 0,
  'WALKING': 0},
 {'Battery Charging': nan,
  'Car Beacon': 0,
  'IN_VEHICLE': 0,
  'Indoor Beacon': 0,
  'Max GPS Accuracy': nan,
  'Max GPS Speed': nan,
  'Max Light Intensity': nan,
  'ON_BICYCLE': 0,
  'ON_FOOT': 0,
  'RUNNING': 0,
  'STILL': 1,
  'TILTING': 0,
  'Time of day': 0,
  'UNKNOWN': 0,
  'WALKING': 0},
 {'Battery Charging': nan,
  'Car Beacon': 0,
  'IN_VEHICLE': 0,
  'Indoor Beacon': 0,
  'Max 

# Data Preprocessing

In [97]:
# Convert features and labels to pandas dataframe format
df_features = pd.DataFrame(features)
df_features['Labels'] = pd.DataFrame(labels)
df_features.to_csv('bharath_features.csv')

In [98]:
pd.set_option('display.max_rows', 1000)
display(df_features)


,Battery Charging,Car Beacon,IN_VEHICLE,Indoor Beacon,Max GPS Accuracy,Max GPS Speed,Max Light Intensity,ON_BICYCLE,ON_FOOT,RUNNING,STILL,TILTING,Time of day,UNKNOWN,WALKING,Labels
0,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
1,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
2,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
3,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
4,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
5,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
6,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
7,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
8,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
9,NaN,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor


In [99]:
df_features['Battery Charging'].interpolate(method='nearest', inplace=True)

df_features['Max GPS Accuracy'].fillna(1000, inplace=True)
#df_features['Max GPS Accuracy'].interpolate(method='nearest', inplace=True)
#df_features['Max GPS Speed'].interpolate(method='nearest', inplace=True)


# df_features.drop('Magnetic Field Variance', axis=1, inplace=True)
df_features = df_features[df_features['Indoor Beacon']!=1]
df_features = df_features[df_features['IN_VEHICLE']!=1]
df_features = df_features[df_features['ON_BICYCLE']!=1]
df_features.drop('Indoor Beacon', axis=1, inplace=True)
df_features.drop('IN_VEHICLE', axis=1, inplace=True)
df_features.drop('ON_BICYCLE', axis=1, inplace=True)
df_features.drop('UNKNOWN', axis=1, inplace=True)
df_features.drop('Max GPS Speed', axis=1, inplace=True)
df_features.dropna(axis=0, inplace=True)

# Separate entire dataframe into different classes
df_incar = df_features[df_features.Labels=='Incar']
df_indoor = df_features[df_features.Labels=='Indoor']
df_outdoor = df_features[df_features.Labels=='Outdoor']

In [100]:
print(df_indoor.shape)
print(df_outdoor.shape)
print(df_incar.shape)
print(df_labels.shape)
print(df_features.shape)


(1872, 11)
(356, 11)
(85, 11)
(708,)
(2313, 11)


In [101]:
# Downsample majority class

df_indoor = resample(df_indoor, 
                     replace=False,   # sample without replacement
                     n_samples=352,   # to match minority class
                     random_state=42) # reproducible results

df_features = pd.concat([df_indoor, df_outdoor]) #df_incar

df_labels = df_features['Labels']
df_features.drop('Labels', axis=1, inplace=True)


# train_features is numpy array
vec = DictVectorizer(sort=False)
train_features = vec.fit_transform(df_features.to_dict('records')).toarray()

# Normalize columns of features with min-max scaler
scaler = MinMaxScaler(copy=False)
train_features = scaler.fit_transform(train_features)

# Encode all features
le = LabelEncoder()
#le.fit(['Incar', 'Indoor', 'Outdoor'])
le.fit(['Indoor', 'Outdoor'])
train_labels = le.transform(df_labels)



In [103]:
display(df_features)
display(scaler.scale_)
display(train_features)

,Battery Charging,Car Beacon,Max GPS Accuracy,Max Light Intensity,ON_FOOT,RUNNING,STILL,TILTING,Time of day,WALKING
18530,0.0,0,1000.000000,546.994995,0,0,1,0,21,0
14104,0.0,0,22.358999,0.000992,0,0,1,0,20,0
17212,0.0,0,1000.000000,416.177002,0,0,1,0,14,0
2590,0.0,0,1000.000000,0.875000,0,0,1,0,14,0
2935,0.0,0,1000.000000,226.388992,0,0,1,0,16,0
9830,0.0,0,1000.000000,29.823990,0,0,0,0,20,0
2347,0.0,0,1000.000000,88.792999,0,0,1,0,13,0
2901,0.0,0,27.257999,733.393005,0,0,0,0,16,0
2656,0.0,0,1000.000000,6.778000,0,0,1,0,14,0
2072,0.0,0,1000.000000,3437.864990,0,0,0,1,11,0


array([  1.00000000e+00,   1.00000000e+00,   7.71139028e-04,
         3.05175790e-05,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   4.34782609e-02,
         1.00000000e+00])

array([[ 0.        ,  0.        ,  0.76865904, ...,  0.        ,
         0.91304348,  0.        ],
       [ 0.        ,  0.        ,  0.01476191, ...,  0.        ,
         0.86956522,  0.        ],
       [ 0.        ,  0.        ,  0.76865904, ...,  0.        ,
         0.60869565,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.76865904, ...,  0.        ,
         0.65217391,  0.        ],
       [ 0.        ,  0.        ,  0.76865904, ...,  0.        ,
         0.69565217,  0.        ],
       [ 0.        ,  0.        ,  0.76865904, ...,  0.        ,
         0.69565217,  0.        ]])

# Training on SVM

## Tuning SVM Parameters

In [ ]:
clf = SVC()
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 
     'kernel' : ['rbf','poly', 'sigmoid'], 
     'probability' : [True],
     'gamma' : [0.1]}
]
grid = GridSearchCV(clf, param_grid, cv=10, scoring='accuracy')
grid.fit(train_features, train_labels)
print("Grid scores on development set:")

means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print("Best parameters:", grid.best_params_)



In [104]:
# 10 fold corss validation
clf = SVC(C=10, kernel='sigmoid')

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    confusionmatrix = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', confusionmatrix)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Micro F1 Score:', f1_score(y_test, y_pred, average='micro'))
    print('Macro F1 Score:', f1_score(y_test, y_pred, average='macro'))
    print('Weighted F1 Score:', f1_score(y_test, y_pred, average='weighted'))
    print('\n')


Confusion Matrix:
 [[27  8]
 [12 24]]
Accuracy: 0.718309859155
Micro F1 Score: 0.718309859155
Macro F1 Score: 0.717806041335
Weighted F1 Score: 0.717638102062


Confusion Matrix:
 [[39  4]
 [11 17]]
Accuracy: 0.788732394366
Micro F1 Score: 0.788732394366
Macro F1 Score: 0.76629361422
Weighted F1 Score: 0.781592782501


Confusion Matrix:
 [[31  6]
 [ 8 26]]
Accuracy: 0.802816901408
Micro F1 Score: 0.802816901408
Macro F1 Score: 0.801834130781
Weighted F1 Score: 0.802423793158


Confusion Matrix:
 [[42  2]
 [10 17]]
Accuracy: 0.830985915493
Micro F1 Score: 0.830985915493
Macro F1 Score: 0.807065217391
Weighted F1 Score: 0.8233312921


Confusion Matrix:
 [[32  5]
 [10 24]]
Accuracy: 0.788732394366
Micro F1 Score: 0.788732394366
Macro F1 Score: 0.786015672092
Weighted F1 Score: 0.787034442945


Confusion Matrix:
 [[28  4]
 [10 29]]
Accuracy: 0.802816901408
Micro F1 Score: 0.802816901408
Macro F1 Score: 0.802777777778
Weighted F1 Score: 0.803051643192


Confusion Matrix:
 [[25  2]
 [14 30]]

In [121]:
# save the model to disk
filename = 'io.model'
pickle.dump(clf, open(filename, 'wb'))

In [114]:
def normalize(feature):
    '''
    Input: feature - 1D numpy array
    Output: normalized feature - 1D numpy array
    '''
    scale = np.array([  1.00000000e+00,   1.00000000e+00,   7.71139028e-04,
         3.05175790e-05,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   4.34782609e-02,
         1.00000000e+00])
    return np.multiply(feature, scale)

In [118]:
def predict_labels(feature_frame):
    '''
    Input: features - panda frame
    Ouput: labels - list
    '''
    df_features['Battery Charging'].interpolate(method='nearest', inplace=True)
    df_features.drop('Max GPS Speed', axis=1, inplace=True)
    df_features['Max GPS Accuracy'].fillna(1000, inplace=True)
    df_features.dropna(axis=0, inplace=True)
    labels = []
    for index, row in feature_frame.iterrows():
        if row['IN_VEHICLE'] or row['Car Beacon']:
            labels.append('Incar')
        elif row['ON_BICYCLE']:
            labels.append('Outdoor')
        elif row['Indoor Beacon']:
            labels.append('Indoor') 
        else:
            # TODO: use classifier to predict
            row.drop('Indoor Beacon', inplace=True)
            row.drop('IN_VEHICLE', inplace=True)
            row.drop('ON_BICYCLE', inplace=True)
            row.drop('UNKNOWN', inplace=True)
            filename = 'bhararh_model.clf'
            model = pickle.load(open(filename, 'rb'))

            y_pred = model.predict([normalize(row.values)])
            if y_pred[0] == 0:
                labels.append('Indoor')
            else:
                labels.append('Outdoor')
    return labels

In [119]:
# Convert features and labels to pandas dataframe format
df_features = pd.DataFrame(features)
ls = predict_labels(df_features)

In [120]:
print(ls)

['Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Outdoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor', 'Indoor'